### HOSPITAIS

#### Libs

In [1]:
import pandas as pd
import plotly.graph_objects as go

DATADIR = '../app/datasets/inputs'
DATAOUT = '../app/datasets/outputs'

#### Base de Leitos

In [24]:
leitos_file_columns = {
    'cd_nome_estab': 'cd_estab',
    'nome_estab': 'estabelecimento', 
    'OBSTETRICIA CIRURGICA': 'obst_cirurg', 
    'OBSTETRICIA CLINICA': 'obst_clinica', 
    'OBSTETRICIA CLINICA': 'obst_clinica', 
    'UTI NEONATAL - TIPO I': 'uti_neo_i', 
    'UTI NEONATAL - TIPO II': 'uti_neo_ii', 
    'UTI NEONATAL - TIPO III': 'uti_neo_iii', 
    'UNIDADE DE CUIDADOS INTERMED NEONATAL CONVENCIONAL': 'uci_neo_conv', 
    'UNIDADE DE CUIDADOS INTERMED NEONATAL CANGURU': 'uci_neo_canguru', 
    # 'total_leitos_obstetricos': 'total_leitos_obstetricos', 
}

df_leitos_obst = pd.read_excel(
    # f'{DATADIR}/leitos_obstetricos.csv', 
    f'{DATADIR}/hospitais_por_leito.xlsx', 
    usecols=leitos_file_columns.keys(),
    # dtype={'LAT,N,11,10': str, 'LONG,N,11,10': 'str'}
).\
    rename(columns=leitos_file_columns)

# df_leitos_obst['cd_estab'] = df_leitos_obst['cd_nm_estabelecimento'].apply(lambda x: x[:7])


df_leitos_obst.head()

,cd_estab,estabelecimento,obst_cirurg,obst_clinica,uti_neo_i,uti_neo_ii,uti_neo_iii,uci_neo_conv,uci_neo_canguru
0,0000019,POLICLINICA DR JAMACI DE MEDEIROS,0,0,0,0,0,0,0
1,0000027,CASA DE SAUDE SANTA HELENA,2,6,0,0,0,0,0
2,0000035,HOSPITAL MENDO SAMPAIO,0,0,0,0,0,0,0
3,0000043,POLICLINICA DR MANUEL GOMES,0,0,0,0,0,0,0
4,0000051,POLICLINICA VICENTE MENDES,0,0,0,0,0,0,0


#### Base de pontos

In [23]:
df_geo_estab = pd.read_csv(
    f'{DATAOUT}/estabelecimentos_saude_pe_pontos.csv', sep=','
    )

df_geo_estab['cd_estab'] = df_geo_estab.cod_estab.astype(str).str.zfill(7)
df_geo_estab.\
    drop(['cod_estab', 'LATITUDE', 'LONGITUDE', 'LEITO_OBST'], axis=1, inplace=True)
df_geo_estab.rename(columns={'X': 'lng', 'Y': 'lat'}, inplace=True)
df_geo_estab.head()

,lng,lat,nome_estab,MUNICIPIO,PARTO2023,GESTAO,cd_estab
0,-35.031554,-8.280487,CASA DE SAUDE SANTA HELENA,CABO DE SANTO AGOSTINHO,SIM,MUNICIPAL,0000027
1,-34.976975,-8.232229,MATERNIDADE PADRE GERALDO LEITE BASTOS,CABO DE SANTO AGOSTINHO,SIM,MUNICIPAL,0000094
2,-35.034854,-8.284423,HOSPITAL SAMARITANO,CABO DE SANTO AGOSTINHO,SIM,MUNICIPAL,0000183
3,-35.034004,-8.282599,HOSPITAL SAO SEBASTIAO,CABO DE SANTO AGOSTINHO,SIM,MUNICIPAL,0000221
4,-34.946315,-8.047435,HOSPITAL DAS CLINICAS,RECIFE,SIM,ESTADUAL,0000396


#### Merging Leitos e Pontos

In [28]:
df_geo_leitos_estab = df_geo_estab[
    ['cd_estab', 'MUNICIPIO', 'PARTO2023', 'GESTAO', 'lat', 'lng']
].merge(df_leitos_obst, how='left', on='cd_estab')

df_geo_leitos_estab.head()

,cd_estab,MUNICIPIO,PARTO2023,GESTAO,lat,lng,estabelecimento,obst_cirurg,obst_clinica,uti_neo_i,uti_neo_ii,uti_neo_iii,uci_neo_conv,uci_neo_canguru
0,0000027,CABO DE SANTO AGOSTINHO,SIM,MUNICIPAL,-8.280487,-35.031554,CASA DE SAUDE SANTA HELENA,2,6,0,0,0,0,0
1,0000094,CABO DE SANTO AGOSTINHO,SIM,MUNICIPAL,-8.232229,-34.976975,MATERNIDADE PADRE GERALDO LEITE BASTOS,16,0,0,0,0,0,0
2,0000183,CABO DE SANTO AGOSTINHO,SIM,MUNICIPAL,-8.284423,-35.034854,HOSPITAL SAMARITANO,5,0,0,0,0,0,0
3,0000221,CABO DE SANTO AGOSTINHO,SIM,MUNICIPAL,-8.282599,-35.034004,HOSPITAL SAO SEBASTIAO,2,2,0,0,0,0,0
4,0000396,RECIFE,SIM,ESTADUAL,-8.047435,-34.946315,HOSPITAL DAS CLINICAS,36,9,0,10,0,5,0


In [31]:
df_geo_leitos_estab.to_parquet(
    f'{DATAOUT}/estabelecimentos_leitos_pontos.parquet.gzip', index=False
    )